<a href="https://colab.research.google.com/github/LucianoClemente40/ChatBot-Diabeticos/blob/main/Calculo_de_Bolus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [36]:
##########################################
# --- Agente 1: Preparação do atendimento --- #
##########################################
topico ="Bolus insulina"
def Atendente(topico, data_de_hoje, valor_glicemico):

    primeiro_atendente = Agent(
        name="atendente_1",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de saúde. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas conhecimento sobre diabetes relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele e em sua qualidade.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo cinco anos antes da data de hoje.
        """,
        description="Agente de saúde especialista em Diabetes",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}\nValor_glicemico: {valor_glicemico}"

    base_conh = call_agent(primeiro_atendente, entrada_do_agente_buscador)
    return base_conh

In [43]:
################################################
# --- Agente 2: Tratamento da hiperglicemia --- #
################################################
def agente_planejador(topico, base_conh, valor_glicemico ):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um assistente de saúde, especialista em diabetes. Com base na lista de
        conhecimentos mais recentes e relevantes buscados, você deve:
        usar a ferramenta de busca do Google (google_search) para aprofundar o
        tema e indicar a dose recomendada de insulina para correção da glicemia (bolus).
        A resposta deve ser: Você precisa tormar (dose idicada) unidade de insulina.
        Sempre concidere o valor glicemico informado pelo paciente como sendo Valor = número mg/dl.
        Se o resultado for decimal < ,5 arredonde para baixo se > arredonde para
        cima, sempre no inteiro mais proxímo.
        Considere faxa ótima de glicemia entre 60 e 100 mg/dl e faça suas indicações para valores
        abaixo e acima disso.
        Considere o FC = 30.
        Seus resposta serão sucintas e objetivas.
        """,
        description="Agente que planeja o bolos de insulina",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nConhecimento_buscados: {base_conh}\nValor_glicemico: {valor_glicemico}"
    # Executa o agente
    plano_de_bolus = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_bolus

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")
topico = "Bolus insulina"

print("🚀 Iniciando o atendimento para calcular sua dose para o bolus de insulina 🚀")

# --- Obter o Tópico do Usuário ---
valor_glicemico = input("❓ Por favor, digite sua glicemia: ")

# Inserir lógica do sistema de agentes ################################################
if not valor_glicemico:
    print("Você precisa informar sua glicemia")
else:
    print(f"Maravilha! Vamos ver quanto de insulina você precisa {valor_glicemico}")

    conhecimento = Atendente(topico, data_de_hoje, valor_glicemico)

    plano_de_bolus = agente_planejador(topico, conhecimento, valor_glicemico)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_bolus))
    print("--------------------------------------------------------------")